# A Tutorial for using Bandits for Travel Page Layout Optimization

- Robert F. Dickerson

Based on [An Efficient Bandit Algorithm for Realtime Multivariate Optimization]() Daniel Hill et. al. (Amazon.com) KDD'17

In [14]:
import pandas as pd
import random
import numpy as np

random.seed(42)

## Page layout problem

Suppose a web page has a static layout containing multiple slots to put content. 

- Slot 1 can contain a: 
    1. weather widget,
    2. list of similar rentals
    3. an advertisement
- Slot 2 can contain a: 
    1. weather widget
    2. advertisement
    3. an instant booking widget
- Slot 3 can contain a:
    1. instant booking widget
    2. advertisement

In this case, the number of potential layouts, $A$, are $3 \times 3 \times 2 = 18$

If we assume each slot can contain $N$ types of content, and there are $D$ slots then there are $N^D$ potential layouts. Quickly becomes a **combinatorial explosion!**

We can add context, $X$, such as time of year or user history so that perhaps certain widgets should be presented more often based on a user's viewing habit.

A feature vector can be formed $B_{A, X}$ combining layout and context.

Let's assume that a reward, $R$, could be that a booking was eventually made. $R \in \{-1, 1\}$

We can store the history $H_t$ of all the layouts users have seen as a tuple, $(A_t, X_t, R_t)$. 

## Generate random simulated data

Let's generate artificially some random combinations of content and assign the value for reward:

- If instant booking is in slot 3 then a booking is $p=0.4$ likely.
- If instant booking is in slot 3 and similar rentals are shown in slot 1 then a booking is $p=0.7$ likely.
- In all other cases, bookings are $p=0.2$ likely

This is highly simplistic example, but it does allow us to represent randomness with a Bernoulli trial. Every time we expose a user to a page layout we draw a random sample that is binary. It demonstrates interaction effects as well.


In [16]:
choices_slot_1 = ["weather", "similar_rentals", "advert"]
choices_slot_2 = ["weather", "advert", "instant_book"]
choices_slot_3 = ["instant_book", "advert"]

# number of random layouts to generate
N = 100


s1 = random.choices(choices_slot_1, k=N)
s2 = random.choices(choices_slot_2, k=N)
s3 = random.choices(choices_slot_3, k=N)

def is_reward(s1, s2, s3):
    
    # draw a uniform random sample
    r = random.random()
    
    p = 0.2
    
    if s3 == 'instant_book':
        p = 0.4
        if s1 == 'similar_rentals':
            # similar rentals widget combined with instant book
            p = 0.7
    
    return 1 if r > p else -1
    

data = {'slot1': s1, 'slot2': s2, 'slot3': s3}
df = pd.DataFrame(data=data)
df['reward'] = df.apply(lambda x: is_reward(x['slot1'], x['slot2'], x['slot3']), axis=1)

display(df)

,slot1,slot2,slot3,reward
0,similar_rentals,weather,advert,1
1,weather,instant_book,advert,1
2,weather,instant_book,advert,1
3,weather,advert,instant_book,1
4,advert,weather,instant_book,-1
...,...,...,...,...
95,similar_rentals,instant_book,advert,1
96,advert,advert,instant_book,1
97,similar_rentals,weather,instant_book,-1
98,advert,instant_book,instant_book,1


In [17]:
def feature_vector(slot1, slot2, slot3):
    """Creates a one-hot binary vector based on the layout"""
    s1 = choices_slot_1.index(slot1)
    s2 = choices_slot_2.index(slot2)
    s3 = choices_slot_3.index(slot3)
    
    sx1 = len(choices_slot_1)
    sx2 = len(choices_slot_2)
    sx3 = len(choices_slot_3)
    
    a = np.zeros(sx1 + sx2 + sx3)
    
    a[s1] = 1
    a[s2 + sx1] = 1
    a[s3 + sx1 + sx2] = 1
    return a
    
    
df['feature'] = df.apply(lambda x: feature_vector(x['slot1'], x['slot2'], x['slot3']), axis=1)

display(df)

,slot1,slot2,slot3,reward,feature
0,similar_rentals,weather,advert,1,"[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]"
1,weather,instant_book,advert,1,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]"
2,weather,instant_book,advert,1,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]"
3,weather,advert,instant_book,1,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0]"
4,advert,weather,instant_book,-1,"[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0]"
...,...,...,...,...,...
95,similar_rentals,instant_book,advert,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]"
96,advert,advert,instant_book,1,"[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0]"
97,similar_rentals,weather,instant_book,-1,"[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]"
98,advert,instant_book,instant_book,1,"[0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]"


In [4]:

from scipy.stats import norm, bernoulli

d = 8
t = 100

mu = np.zeros(d)
sigma = np.ones(d)

def v(t):
    return norm.pdf(t)

def w(t):
    return v(t) * (v(t) + t)

for i in range(t):
    row = df.iloc[i]
    feature = row['feature']
    reward = row['reward']
    
    beta = 1
    
    variance = beta**2 + feature*sigma
        
    mu = mu + reward * feature * sigma/variance * v(reward*feature*mu/sigma)
    sigma = sigma * (1 - feature * sigma/variance * w(reward * feature * mu/variance))
    
#     print('mu: {}'.format(mu))
#     print('sigma: {}'.format(sigma))

In [5]:
import matplotlib.pyplot as plt

for i in range(8):

    x = np.linspace(norm.ppf(0.01), norm.ppf(0.99), 100)

    y = norm.pdf(x, loc=mu[i], scale=sigma[i])

    ax = plt.subplot(4, 2, i+1)
    ax.plot(x, y, 'r-', label=str(i))
    ax.set_title(r'$W_{}$'.format(i))


plt.tight_layout()
plt.show()

<Figure size 640x480 with 8 Axes>

## Thompson Sampling

solve $\arg \max_A B^T_{A, X} W$

- for all $t = 1, \ldots, T$ do
    - receive context $X_t$
    - sample $W_T$ from the posterior $P(W|H_{t-1})$
    - select $A_t = \arg \max_A B^T_{A, X_t} W_t$
    - display layout $A_t$ and observe reward $R_t$
    - update $H_t = H_{t-1} \cup (A_t, R_t, X_t)$

In [19]:
w_tilde = np.random.normal(mu, sigma)

b = [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]

np.sum(b*w_tilde)

2.3415083823241947

In [55]:
df

,slot1,slot2,slot3,reward,feature
0,similar_rentals,weather,advert,1,"[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]"
1,weather,instant_book,advert,1,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]"
2,similar_rentals,instant_book,advert,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]"
3,advert,advert,advert,1,"[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0]"
4,similar_rentals,weather,instant_book,-1,"[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]"
5,weather,advert,instant_book,-1,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0]"
6,advert,weather,instant_book,1,"[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0]"
7,similar_rentals,advert,advert,1,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0]"
8,weather,advert,advert,1,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0]"
9,weather,instant_book,advert,1,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]"
